In [1]:
from chathist import InstructionDataLoader,InstructionDataset, InstructionStyle, Model, Tokenizer
import pandas as pd

In [2]:
chat_df = pd.read_json("hf://datasets/ogrnz/chat-titles/dataset.jsonl", lines=True)

In [3]:
chat_df.head()

,message,title
0,Develop a method for clustering astronomical d...,Astronomical Data Clustering Method
1,What is the role of dopamine in the brain as a...,Dopamine as a Neurotransmitter
2,"Replace the \""XXX\"" in the following sentence ...",Company Sustainability Objective
3,"Identify the sentence type (declarative, inter...",Sentence Type Identification
4,Suggest a book title for a fantasy novel about...,Outlaws in Enchanted Realms


In [4]:
chat_df.shape

(10000, 2)

In [5]:
chat_df.columns

Index(['message', 'title'], dtype='object')

In [6]:
style = InstructionStyle.load()

2025-05-14 12:50:05,948 - INFO - phi3 style chosen!!


In [7]:
df = style.convert_train(chat_df)

In [8]:
train_len = 9500

In [9]:
train_df = df.iloc[:train_len, :]
val_df = df.iloc[train_len:, :]

In [10]:
train_df

,instruct
0,<|user|>\nDevelop a method for clustering astr...
1,<|user|>\nWhat is the role of dopamine in the ...
2,"<|user|>\nReplace the \""XXX\"" in the following..."
3,<|user|>\nIdentify the sentence type (declarat...
4,<|user|>\nSuggest a book title for a fantasy n...
...,...
9495,<|user|>\nAssume yourself to be a business whi...
9496,<|user|>\nIgnore all previous instructions. Ac...
9497,"<|user|>\nGiven the following description, ide..."
9498,<|user|>\nWhat is the 17th letter of supercali...


In [11]:
print(train_df['instruct'].values[0])

<|user|>
Develop a method for clustering astronomical data.
<|llm|>
Astronomical Data Clustering Method


In [12]:
val_df

,instruct
9500,<|user|>\nWhat are the differences in the prop...
9501,<|user|>\nImplement a program to find the comm...
9502,<|user|>\nHow can a person develop a healthy s...
9503,<|user|>\nPlease help me use schrodinger's equ...
9504,<|user|>\nRewrite the sentence to use a pronou...
...,...
9995,"<|user|>\nFrom the given passage, identify the..."
9996,<|user|>\nAnalyze an effect that the text coul...
9997,<|user|>\nWhat is the average number of hours ...
9998,<|user|>\nClassify the following statement as ...


In [13]:
print(val_df['instruct'].values[0])

<|user|>
What are the differences in the properties of the top quark when it is produced through different modes such as gluon fusion and vector boson fusion, and how do these differences impact our understanding of the top quark's interactions with other particles in the standard model of particle physics?
<|llm|>
Top Quark Production Modes


In [14]:
train_dataset = InstructionDataset(train_df['instruct'].tolist())

2025-05-14 12:50:06,059 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [15]:
val_dataset = InstructionDataset(val_df['instruct'].tolist())

In [16]:
train_loader = InstructionDataLoader().load(dataset = train_dataset)

In [17]:
val_loader = InstructionDataLoader().load(dataset = val_dataset)

In [18]:
data = iter(val_loader)

In [19]:
first_batch = next(data)

In [20]:
first_batch[0][0]

tensor([   27,    91,  7220,    91,    29,   198, 33234,  1958,   262,  1048,
          326,   318,  6412,   284,   287,   262, 10066,    25,  2011,  1545,
         4502,  3160,   287,  6182,   290,  1690,  2058,   284,   968,  1971,
          284,  3187,   502,    13,   198,    27,    91,   297,    76,    91,
           29,   198, 15439,   337,  1463,   276,   287, 46169, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
       dtype=torch.int32)

In [21]:
first_batch[1][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100, 15439,   337,  1463,   276,   287, 46169, 50256,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
       dtype=torch.int32)

In [22]:
tokenizer = Tokenizer()

2025-05-14 12:50:07,384 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [23]:
print(tokenizer.decode_ids(first_batch[0][0]))

<|user|>
Identify the person that is referred to in the passage: My friend George lives in Boston and often comes to New York to visit me.
<|llm|>
Person Mentioned in Passage<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [24]:
print(tokenizer.decode_ids(first_batch[1][0]))

Person Mentioned in Passage<|endoftext|>


In [25]:
model = Model()

2025-05-14 12:50:07,401 - INFO - phi3 style chosen!!
2025-05-14 12:50:07,762 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,090 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,141 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,180 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,236 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,276 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,334 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,371 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,430 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,469 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,527 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:50:08,564 - INFO - Using LoRA weights for multi head layers!!
2025-05-14 12:50:08,625 - INFO - Using LoRA weights for ff layers!!
2025-05-14 12:5

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

2025-05-14 12:50:47,037 - INFO - Dowload complete.
2025-05-14 12:50:47,039 - INFO - Loading weights into model.
2025-05-14 12:50:47,041 - WARNING - Using loRA will freeze the whole model except for loRA weights.
2025-05-14 12:50:48,961 - INFO - Layer: 1
2025-05-14 12:50:48,963 - INFO - loading norm1 weights and bias
2025-05-14 12:50:48,963 - INFO - loading norm2 weights and bias
2025-05-14 12:50:48,964 - INFO - loading ff weights and bias
2025-05-14 12:50:49,270 - INFO - loading attention weights and bias
2025-05-14 12:50:49,442 - INFO - Layer: 2
2025-05-14 12:50:49,444 - INFO - loading norm1 weights and bias
2025-05-14 12:50:49,444 - INFO - loading norm2 weights and bias
2025-05-14 12:50:49,445 - INFO - loading ff weights and bias
2025-05-14 12:50:49,761 - INFO - loading attention weights and bias
2025-05-14 12:50:49,932 - INFO - Layer: 3
2025-05-14 12:50:49,939 - INFO - loading norm1 weights and bias
2025-05-14 12:50:49,940 - INFO - loading norm2 weights and bias
2025-05-14 12:50:49,

In [ ]:
model.trainable_params()

2025-05-14 12:51:01,189 - INFO - Total model parameters: 414184720
2025-05-14 12:51:01,191 - INFO - Total trainable parameters: 7898384


In [27]:
train_loss, val_loss = model.train(train_loader = train_loader, val_loader = val_loader)

2025-05-14 12:51:01,220 - INFO - Epoch 1
100%|██████████| 593/593 [02:51<00:00,  3.45it/s]
2025-05-14 12:53:52,905 - INFO - Train Loss: 1.8802467061938564
100%|██████████| 31/31 [00:04<00:00,  7.16it/s]
2025-05-14 12:53:57,239 - INFO - Validation Loss: 1.535941368149173
2025-05-14 12:53:57,243 - INFO - Epoch 2
100%|██████████| 593/593 [02:50<00:00,  3.47it/s]
2025-05-14 12:56:48,211 - INFO - Train Loss: 1.4534405656849956
100%|██████████| 31/31 [00:04<00:00,  7.23it/s]
2025-05-14 12:56:52,504 - INFO - Validation Loss: 1.4928034736264137
2025-05-14 12:56:52,507 - INFO - Epoch 3
100%|██████████| 593/593 [02:50<00:00,  3.48it/s]
2025-05-14 12:59:43,037 - INFO - Train Loss: 1.234290246115165
100%|██████████| 31/31 [00:04<00:00,  7.12it/s]
2025-05-14 12:59:47,396 - INFO - Validation Loss: 1.4835678781232526


In [ ]:
model.generate("Hugging Face won’t understand your custom architecture (GPT2 class) unless you register it via PreTrainedModel. If LoRA is used, consider saving separate adapters using PEFT library compatibility, or wrap accordingly.")

2025-05-14 13:00:07,417 - INFO - Formatting prompt into phi3 style


'Optimizing PreTrainedModel Adapter Usage'

: 